# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [4]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)


['thompson',
 'mackay',
 'saint george',
 'rafai',
 'cape town',
 'manta',
 'ancud',
 'busselton',
 'pontianak',
 'dikson',
 'faanui',
 'rikitea',
 'jamestown',
 'kapaa',
 'ushuaia',
 'georgetown',
 'victoria',
 'castro',
 'vao',
 'zlobin',
 'tasiilaq',
 'atuona',
 'mar del plata',
 'puerto ayora',
 'kroya',
 'new norfolk',
 'kenai',
 'changji',
 'east london',
 'turukhansk',
 'road town',
 'barentsburg',
 'abu kamal',
 'kashi',
 'hobart',
 'saint-philippe',
 'sitka',
 'lebu',
 'den helder',
 'barrow',
 'yellowknife',
 'chokurdakh',
 'mataura',
 'ximei',
 'belushya guba',
 'ponta do sol',
 'nenjiang',
 'rosetown',
 'hasaki',
 'burica',
 'nikolskoye',
 'leningradskiy',
 'clifton springs',
 'san patricio',
 'cicevac',
 'belyy yar',
 'torbay',
 'kavieng',
 'galveston',
 'souillac',
 'tuktoyaktuk',
 'ribeira grande',
 'montrose',
 'wanning',
 'taolanaro',
 'geraldton',
 'sentyabrskiy',
 'vallenar',
 'biltine',
 'raudeberg',
 'solnechnyy',
 'arraial do cabo',
 'cayenne',
 'la grande',
 'sim

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [5]:
testcity= "tokyo"
urlBase = "https://api.openweathermap.org/data/2.5/weather?q="
urlappid = "&appid="
urlUnits = "&units=imperial"

In [15]:
latList = []
lonList = []
maxTempList = []
humList = []
cloudcovList = []
windList = []
countryList = []
datetimeList = []
cityIndex = 0
cityExceptList = []
cityExceptIndList = []

for city in cities:
    # Create try/except to compensate for errors
    try:
        print(f'Searching for city at index {cityIndex}, "{city}"')
        fullRequestUrl = f'{urlBase}{city}{urlappid}{weather_api_key}{urlUnits}'
        jsonObj = requests.get(fullRequestUrl).json()
        #Pull data from json response into lists
        latList.append(jsonObj["coord"]["lat"])
        lonList.append(jsonObj["coord"]["lon"])
        maxTempList.append(jsonObj["main"]["temp_max"])
        humList.append(jsonObj["main"]["humidity"])
        cloudcovList.append(jsonObj["clouds"]["all"])
        windList.append(jsonObj["wind"]["speed"])
        countryList.append(jsonObj["sys"]["country"])
        datetimeList.append(jsonObj["timezone"])
    except:
        print(f'        ...city not found. Recording and resuming.')
        cityExceptList.append(city)
        cityExceptIndList.append(cityIndex)

    cityIndex += 1
    time.sleep(1)
print(f'Requests complete!')

Searching for city at index 0, "thompson"
Searching for city at index 1, "mackay"
Searching for city at index 2, "saint george"
Searching for city at index 3, "rafai"
Searching for city at index 4, "cape town"
Searching for city at index 5, "manta"
Searching for city at index 6, "ancud"
Searching for city at index 7, "busselton"
Searching for city at index 8, "pontianak"
Searching for city at index 9, "dikson"
Searching for city at index 10, "faanui"
Searching for city at index 11, "rikitea"
Searching for city at index 12, "jamestown"
Searching for city at index 13, "kapaa"
Searching for city at index 14, "ushuaia"
Searching for city at index 15, "georgetown"
Searching for city at index 16, "victoria"
Searching for city at index 17, "castro"
Searching for city at index 18, "vao"
Searching for city at index 19, "zlobin"
Searching for city at index 20, "tasiilaq"
Searching for city at index 21, "atuona"
Searching for city at index 22, "mar del plata"
Searching for city at index 23, "puer

Searching for city at index 174, "pevek"
Searching for city at index 175, "careiro da varzea"
Searching for city at index 176, "provideniya"
Searching for city at index 177, "vanimo"
Searching for city at index 178, "cidreira"
Searching for city at index 179, "airai"
Searching for city at index 180, "pangnirtung"
Searching for city at index 181, "arlit"
Searching for city at index 182, "sabak"
Searching for city at index 183, "rio grande"
Searching for city at index 184, "mezen"
Searching for city at index 185, "kahului"
Searching for city at index 186, "bow island"
Searching for city at index 187, "birin"
Searching for city at index 188, "ranau"
Searching for city at index 189, "tome-acu"
        ...city not found. Recording and resuming.
Searching for city at index 190, "nhulunbuy"
Searching for city at index 191, "santa maria"
Searching for city at index 192, "moscavide"
Searching for city at index 193, "iracoubo"
Searching for city at index 194, "keuruu"
Searching for city at index

Searching for city at index 337, "snezhnogorsk"
Searching for city at index 338, "vestmanna"
Searching for city at index 339, "port macquarie"
Searching for city at index 340, "san quintin"
Searching for city at index 341, "soyo"
Searching for city at index 342, "inderborskiy"
        ...city not found. Recording and resuming.
Searching for city at index 343, "ambon"
Searching for city at index 344, "pakxan"
Searching for city at index 345, "tanjong sepat"
        ...city not found. Recording and resuming.
Searching for city at index 346, "necochea"
Searching for city at index 347, "amot"
Searching for city at index 348, "nanortalik"
Searching for city at index 349, "naryan-mar"
Searching for city at index 350, "taoudenni"
Searching for city at index 351, "flagstaff"
Searching for city at index 352, "krasnoselkup"
Searching for city at index 353, "werda"
Searching for city at index 354, "saint-pierre"
Searching for city at index 355, "mossendjo"
Searching for city at index 356, "ust-ka

Searching for city at index 503, "dingle"
Searching for city at index 504, "ust-maya"
Searching for city at index 505, "jamsa"
Searching for city at index 506, "margate"
Searching for city at index 507, "sarahan"
Searching for city at index 508, "anadyr"
Searching for city at index 509, "isangel"
Searching for city at index 510, "kungurtug"
Searching for city at index 511, "calatayud"
Searching for city at index 512, "itoman"
Searching for city at index 513, "dongsheng"
Searching for city at index 514, "touros"
Searching for city at index 515, "tenenkou"
Searching for city at index 516, "arcachon"
Searching for city at index 517, "marsa matruh"
Searching for city at index 518, "groningen"
Searching for city at index 519, "luau"
Searching for city at index 520, "honiara"
Searching for city at index 521, "kayerkan"
Searching for city at index 522, "nagaur"
Searching for city at index 523, "rockport"
Searching for city at index 524, "conde"
Searching for city at index 525, "ayan"
Searchin

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [18]:
#title capitalize all cities for formatting
cleanCityList = []
for cityerror in cityExceptList:
    cities.remove(cityerror)
    
cities





#remove items not found

ValueError: list.remove(x): x not in list

In [19]:
cityTitleList = []
for city in cities:
    cityTitleList.append(city.title())



['Thompson',
 'Mackay',
 'Saint George',
 'Rafai',
 'Cape Town',
 'Manta',
 'Ancud',
 'Busselton',
 'Pontianak',
 'Dikson',
 'Faanui',
 'Rikitea',
 'Jamestown',
 'Kapaa',
 'Ushuaia',
 'Georgetown',
 'Victoria',
 'Castro',
 'Vao',
 'Zlobin',
 'Tasiilaq',
 'Atuona',
 'Mar Del Plata',
 'Puerto Ayora',
 'Kroya',
 'New Norfolk',
 'Kenai',
 'Changji',
 'East London',
 'Turukhansk',
 'Road Town',
 'Abu Kamal',
 'Kashi',
 'Hobart',
 'Saint-Philippe',
 'Sitka',
 'Lebu',
 'Den Helder',
 'Barrow',
 'Yellowknife',
 'Chokurdakh',
 'Mataura',
 'Ximei',
 'Ponta Do Sol',
 'Nenjiang',
 'Rosetown',
 'Hasaki',
 'Nikolskoye',
 'Leningradskiy',
 'Clifton Springs',
 'San Patricio',
 'Cicevac',
 'Belyy Yar',
 'Torbay',
 'Kavieng',
 'Galveston',
 'Souillac',
 'Tuktoyaktuk',
 'Ribeira Grande',
 'Montrose',
 'Wanning',
 'Geraldton',
 'Vallenar',
 'Biltine',
 'Raudeberg',
 'Solnechnyy',
 'Arraial Do Cabo',
 'Cayenne',
 'La Grande',
 'Simbahan',
 'Priargunsk',
 'Dumas',
 'San Cristobal',
 'Avarua',
 'Pisco',
 'Pu

In [20]:

citiesdict = {
    "City Name":cityTitleList,
    "City Latitude":latList,
    "City Longitude":lonList,
    "Max Temperature":maxTempList,
    "Humidity":humList,
    "Cloud Coverage":cloudcovList,
    "Wind Speed":windList,
    "City Country":countryList,
    "City Datetime":datetimeList
}
citiesdf = pd.DataFrame(citiesdict)



citiesdf

,City Name,City Latitude,City Longitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed,City Country,City Datetime
0,Thompson,55.74,-97.86,8.60,84,90,13.87,CA,-21600
1,Mackay,-21.15,149.20,78.80,88,75,9.17,AU,36000
2,Saint George,37.10,-113.58,39.99,80,90,1.90,US,-25200
3,Rafai,4.95,23.92,69.89,53,100,1.88,CF,3600
4,Cape Town,-33.93,18.42,62.01,82,75,5.82,ZA,7200
...,...,...,...,...,...,...,...,...,...
565,Saint-Francois,46.42,3.91,42.80,70,90,10.29,FR,3600
566,Zhigansk,66.77,123.37,-10.34,88,100,19.89,RU,32400
567,Nambour,-26.63,152.95,82.99,79,100,1.99,AU,36000
568,Nizhneivkino,58.19,49.52,-10.37,90,5,2.04,RU,10800


In [21]:


citiesdf.to_csv("citiesdfOutput.csv", index=True, header=True)


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression